In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import time

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

import cv2

In [1]:
print('test')

test


In [4]:
# read training images
DATADIR = "./train/"
#CATEGORIES = os.listdir(DATADIR)
CATEGORIES = ["Closed", "Open"]
IMG_SIZE = 100

training_data = []
for category in CATEGORIES:
        path = os.path.join(DATADIR, category) # path to cats or dogs dir
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE) # colour is not specific in this use-case
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

random.shuffle(training_data)

X = [] # features
y = [] # labels

for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # -1 is default, last 1 for grey scale
y = np.array(y)
y = to_categorical(y)

NameError: name 'os' is not defined

In [3]:
batch_size = 64
epochs = 20
num_classes = 4

# compare different architectures
dense_layers = [1]
layer_sizes = [16, 32]
conv_layers = [3, 5]

model = [Sequential() for i in range(len(dense_layers)*len(layer_sizes)*len(conv_layers))]
idx = 0

for dense_layer in dense_layers: 
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            # analyse model
            NAME = 'Drowsiness_Detection_Model_{}_{}_conv_{}_nodes_{}_dense_{}'.format(idx, conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            # input layer
            model[idx].add(Conv2D(layer_size, (3,3), input_shape = X.shape[1:]))
            model[idx].add(Activation('relu'))
            model[idx].add(MaxPooling2D(pool_size = (2,2))) 
            
            # hidden convulational layers (='filters')
            for l in range(conv_layer-1):
                model[idx].add(Conv2D(layer_size, (3,3)))
                model[idx].add(Activation('relu'))
                model[idx].add(MaxPooling2D(pool_size = (2,2)))
            
            # hidden dense (=fully connected) layers
            model[idx].add(Flatten())
            for l in range(dense_layer):
                model[idx].add(Dense(layer_size))
                model[idx].add(Activation('relu'))

            # output layer
            model[idx].add(Dense(len(CATEGORIES)))
            model[idx].add(Activation('sigmoid'))

            model[idx].compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])

            model[idx].fit(X, y, batch_size=4, epochs=5, validation_split = 0.2) # appropriate batch size depends on size of data set
            idx+=1

Drowsiness_Detection_Model_0_3_conv_16_nodes_1_dense_1653480345
Train on 1161 samples, validate on 291 samples
Epoch 1/5
1161/1161 [==============================] - 3s 3ms/sample - loss: 0.6481 - accuracy: 0.6593 - val_loss: 0.1391 - val_accuracy: 0.9502
Epoch 2/5
1161/1161 [==============================] - 2s 2ms/sample - loss: 0.1422 - accuracy: 0.9526 - val_loss: 0.2243 - val_accuracy: 0.9347
Epoch 3/5
1161/1161 [==============================] - 3s 2ms/sample - loss: 0.1183 - accuracy: 0.9604 - val_loss: 0.1543 - val_accuracy: 0.9536
Epoch 4/5
1161/1161 [==============================] - 2s 2ms/sample - loss: 0.1172 - accuracy: 0.9612 - val_loss: 0.2086 - val_accuracy: 0.9485
Epoch 5/5
1161/1161 [==============================] - 2s 2ms/sample - loss: 0.0875 - accuracy: 0.9750 - val_loss: 0.0971 - val_accuracy: 0.9605
Drowsiness_Detection_Model_1_5_conv_16_nodes_1_dense_1653480358
Train on 1161 samples, validate on 291 samples
Epoch 1/5
1161/1161 [==============================] 

In [ ]:
model[0].save('drowsinessEyeCnn.model')

In [12]:
cap = cv2.VideoCapture(0)
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
while cap.isOpened():
    success, image = cap.read()
    
    eyes = eye_cascade.detectMultiScale(image, minSize=(40,40), maxSize=(50,50))
    grayFrame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    left = True
    for (ex,ey,ew,eh) in eyes:
        if left:
            left_eye = grayFrame[ey : ey + eh, ex : ex + ew]
            left_eye = cv2.resize(left_eye, (IMG_SIZE, IMG_SIZE))
            cv2.rectangle(image, (ex,ey), (ex+ew, ey+eh), (0,255,0), 6)
            cv2.imshow('Left Eye', left_eye)
            left_eye = np.array(left_eye).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # -1 is default, last 1 for grey scale
            pred = model[1].predict(left_eye)
            left = False
        else:
            right_eye = grayFrame[ey : ey + eh, ex : ex + ew]
            right_eye = cv2.resize(right_eye, (IMG_SIZE, IMG_SIZE))
            cv2.rectangle(image, (ex,ey), (ex+ew, ey+eh), (0,255,0), 6)
            cv2.imshow('Right Eye', right_eye)
            right_eye = np.array(right_eye).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # -1 is default, last 1 for grey scale
            pred = model[1].predict(right_eye)
            left = True
            
        if(pred[0,0] > 0.9): 
            print('eyes closed')
    
    
    cv2.imshow('Video Stream', image)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes closed
eyes

KeyboardInterrupt: 